In [6]:
%matplotlib inline

import numpy as np
import pandas as pd
import random
import requests as rq
import sys
import io
from bs4 import BeautifulSoup
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, LSTM
from keras.optimizers import RMSprop
from collections import Counter
from datetime import datetime
import keras
import keras.callbacks
from keras.callbacks import TensorBoard
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [11]:
print("Drunk Philosophers")

data_raw = pd.read_csv("./data/philosophy_data.csv")
data_raw = data_raw.drop(['sentence_spacy', 'original_publication_date', 'corpus_edition_date', 'sentence_length', 'sentence_lowered', 'lemmatized_str'], axis=1)
philosophers = list(dict.fromkeys(data_raw['author'].tolist()))

selected_data = data_raw.loc[data_raw['author'] == "Plato"]
text = ' '.join(selected_data['sentence_str'].tolist())

print(text[:2000])

Drunk Philosophers
 What's new, Socrates, to make you leave your usual haunts in the Lyceum and spend your time here by the king archon's court? Surely you are not prosecuting anyone before the king archon as I am? The Athenians do not call this a prosecution but an indictment, Euthyphro. What is this you say? Someone must have indicted you, for you are not going to tell me that you have indicted someone else. But someone else has indicted you? I do not really know him myself, Euthyphro. He is apparently young and unknown. They call him Meletus, I believe. He belongs to the Pitthean deme, if you know anyone from that deme called Meletus, with long hair, not much of a beard, and a rather aquiline nose. I don't know him, Socrates. What charge does he bring against you? A not ignoble one I think , for it is no small thing for a young man to have knowledge of such an important subject. He says he knows how our young men are corrupted and who corrupts them. He is likely to be wise, and when

In [13]:
chars = sorted(list(set(text)))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

seqlen = 5
step = seqlen
sentences = []
for i in range(0, len(text) - seqlen - 1, step):
    sentences.append(text[i: i + seqlen + 1])

x = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), seqlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, (char_in, char_out) in enumerate(zip(sentence[:-1], sentence[1:])):
        x[i, t, char_indices[char_in]] = 1
        y[i, t, char_indices[char_out]] = 1


model = Sequential()
model.add(LSTM(128, input_shape=(seqlen, len(chars)), return_sequences=True))
model.add(Dense(len(chars), activation='softmax'))

model.compile(
    loss='categorical_crossentropy',
    optimizer=RMSprop(learning_rate=0.01),
    metrics=['categorical_crossentropy', 'accuracy']
)

def sample(preds, temperature=1.0):
    """Helper function to sample an index from a probability array."""
    preds = np.asarray(preds).astype('float64')
    preds = np.exp(np.log(preds) / temperature)  # softmax
    preds = preds / np.sum(preds)                #
    probas = np.random.multinomial(1, preds, 1)  # sample index
    return np.argmax(probas)                     #


def on_epoch_end(epoch, _):
    """Function invoked at end of each epoch. Prints generated text."""
    print()
    print('----- Generating text after Epoch: %d' % epoch)

    start_index = random.randint(0, len(text) - seqlen - 1)
    
    # Q5: What does diversity do?
    diversity = 0.5
    print('----- diversity:', diversity)

    generated = ''
    sentence = text[start_index: start_index + seqlen]
    generated += sentence
    print('----- Generating with seed: "' + sentence + '"')
    sys.stdout.write(generated)

    for i in range(400):
        x_pred = np.zeros((1, seqlen, len(chars)))
        for t, char in enumerate(sentence):
            x_pred[0, t, char_indices[char]] = 1.

        preds = model.predict(x_pred, verbose=0)
        next_index = sample(preds[0, -1], diversity)
        next_char = indices_char[next_index]

        sentence = sentence[1:] + next_char

        sys.stdout.write(next_char)
        sys.stdout.flush()
    print()

print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

model.fit(x, y,
          batch_size=128,
          epochs=1,
          callbacks=[print_callback])

model.save("models/Plato")

/Users/jackbodine/opt/anaconda3/envs/DeepLearning_Env/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  # This is added back by InteractiveShellApp.init_path()
/Users/jackbodine/opt/anaconda3/envs/DeepLearning_Env/lib/python3.7/site-packages/ipykernel_launcher.py:12: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#d

Epoch 1/1


2022-04-19 11:51:24.656392: E tensorflow/core/grappler/optimizers/dependency_optimizer.cc:697] Iteration = 0, topological sort failed with message: The graph couldn't be sorted in topological order.
2022-04-19 11:51:24.662182: E tensorflow/core/grappler/optimizers/dependency_optimizer.cc:697] Iteration = 1, topological sort failed with message: The graph couldn't be sorted in topological order.
2022-04-19 11:51:24.677877: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:502] model_pruner failed: Invalid argument: MutableGraphView::MutableGraphView error: node 'loss_2/dense_3_loss/categorical_crossentropy/weighted_loss/concat' has self cycle fanin 'loss_2/dense_3_loss/categorical_crossentropy/weighted_loss/concat'.
2022-04-19 11:51:24.696539: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:502] remapper failed: Invalid argument: MutableGraphView::MutableGraphView error: node 'loss_2/dense_3_loss/categorical_crossentropy/weighted_loss/concat' has self cycle fanin 'loss_2/d

889615/889615 [==============================] - 395s 443us/step - loss: 1.7937 - categorical_crossentropy: 1.7937 - accuracy: 0.4580

----- Generating text after Epoch: 0
----- diversity: 0.5
----- Generating with seed: " trib"
 tribe one it is soul s or same to then the for that was accept then it one who case the good of the same to say, and large of the many the good and a man who was an accounter to be a state of the law any other of the soul so the other that if the greater is it is it is a particular with who case and poets the only in his one are they are a man which the many. It see the one doubles to the same present
